&nbsp;

# Audio Analysis Basics

This notebook is inspired from a tutorial that I was following for pyaudio

### Imports

In [65]:
import pyaudio
import struct
import numpy as np
import matplotlib.pyplot as plt
import wave

In [18]:
%matplotlib inline

### Basic wave plotting

In [3]:
chunk = 1024 * 4
data_format = pyaudio.paInt16
channels = 1
rate = 44100

In [8]:
pyaudio_obj = pyaudio.PyAudio()

stream = pyaudio_obj.open(
    format=data_format,
    channels = channels,
    rate = rate,
    input = True,
    output = True,
    frames_per_buffer=chunk
)

In [10]:
data = stream.read(chunk)

In [12]:
len(data)

8192

In [24]:
data_int = np.array(struct.unpack(str(2  * chunk) + "B", data), dtype="b") + 127
data_int

array([  96,  126, -101, ...,  123,  101,  125], dtype=int8)

In [33]:
fig, ax = plt.subplots()
ax.plot(data_int, '-')
plt.show()

### Real-time wave plotting

In [44]:
%matplotlib tk
# %matplotlib inline

In [51]:
fig, ax = plt.subplots()

x = np.arange(0, 2*chunk, 2)
line, = ax.plot(x, np.random.rand(chunk))

ax.set_ylim(0, 255)
ax.set_xlim(0, chunk)

while True:
    data = stream.read(chunk)
    data_int = np.array(struct.unpack(str(2  * chunk) + 'B', data), dtype="b")[::2] #+ 127
    line.set_ydata(data_int)
    fig.canvas.draw()
    fig.canvas.flush_events()

TclError: 

### Saving the mic input to drive

In [61]:
p = pyaudio.PyAudio()

stream = p.open(format=data_format,
                channels=channels,
                rate=rate,
                input=True,
                frames_per_buffer=chunk
)

In [62]:
record_for_seconds = 5
output_filename = "output.wav"

In [63]:
print("* recording")

frames = []

for i in range(0, int(rate / chunk * record_for_seconds)):
    data = stream.read(chunk)
    frames.append(data)

print("* done recording")
stream.stop_stream()
stream.close()
p.terminate()

* recording
* done recording


In [67]:
wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
wf.setnchannels(channels)
wf.setsampwidth(p.get_sample_size(data_format))
wf.setframerate(rate)
wf.writeframes(b''.join(frames))
wf.close()

Exception ignored in: <function Wave_write.__del__ at 0x7f6d1575b1f0>
Traceback (most recent call last):
  File "/usr/lib/python3.8/wave.py", line 327, in __del__
    self.close()
  File "/usr/lib/python3.8/wave.py", line 445, in close
    self._ensure_header_written(0)
  File "/usr/lib/python3.8/wave.py", line 463, in _ensure_header_written
    raise Error('# channels not specified')
wave.Error: # channels not specified


### Fast Fourier Transform on mic input